## Residual Network 구현 및 학습

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## Residual Unit 구현

In [9]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')
            
    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h

## Residual Layer 구현

In [10]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        # 16, [32, 32, 32, 32]
        self.sequence = list()
        for f_in, f_out in zip([filter_in]+list(filters), filters):
            # [16] + [32, 32, 32]
            # zip([16,32,32,32], [32, 32 ,32])
            self.sequence.append(ResidualUnit(f_in, f_out,kernel_size))
            
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

## 모델 정의

In [16]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu') # 28x28x8
        
        self.res1 = ResnetLayer(8, (16,16),(3,3))# 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2,2)) #14X14X16
        
        self.res2 = ResnetLayer(8, (32,32),(3,3))# 14X14X32
        self.pool2 = tf.keras.layers.MaxPool2D((2,2)) #7X7X32
        
        self.res3 = ResnetLayer(8, (64,64),(3,3))# 7X7X64
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.res1(x,training=training)
        x = self.pool1(x)
        x = self.res2(x,training=training)
        x = self.pool2(x)
        x = self.res3(x,training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 학습, 테스트 루프 정의

In [17]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [18]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [19]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [20]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.1413198858499527, Accuracy: 96.03333282470703, Test Loss: 0.0686977282166481, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 0.06713824719190598, Accuracy: 98.20832824707031, Test Loss: 0.05676257982850075, Test Accuracy: 98.27999877929688
Epoch 3, Loss: 0.04923100769519806, Accuracy: 98.59166717529297, Test Loss: 0.0674724355340004, Test Accuracy: 98.33999633789062
Epoch 4, Loss: 0.04008091986179352, Accuracy: 98.88500213623047, Test Loss: 0.045307353138923645, Test Accuracy: 98.68000030517578
Epoch 5, Loss: 0.03110685385763645, Accuracy: 99.13166809082031, Test Loss: 0.03907785937190056, Test Accuracy: 98.83999633789062
Epoch 6, Loss: 0.031503140926361084, Accuracy: 99.13166809082031, Test Loss: 0.03405801206827164, Test Accuracy: 99.1500015258789
Epoch 7, Loss: 0.02577386610209942, Accuracy: 99.26000213623047, Test Loss: 0.04154163971543312, Test Accuracy: 98.94000244140625
Epoch 8, Loss: 0.023320432752370834, Accuracy: 99.31999969482422, Test Loss: 0.03640304505825